## 1. import

In [2]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from tqdm import tqdm
from dotenv import load_dotenv
load_dotenv()


c:\Users\Playdata\AppData\Local\anaconda3\envs\vectordb_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

## 2. pdf load

In [ ]:

# 피디에프 로드
path = './하나손해보험 약관파일/1_자동차보험/1_3_공동인수 자동차보험(개인용)_pdf.pdf'
loader = PyPDFLoader(path)
pages = loader.load()

# 텍스트 나누기
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)
documents = text_splitter.split_documents(pages)

# 문장 추출
sentences = [doc.page_content for doc in documents]

len(sentences)

## 3. embedding

In [ ]:
# 허깅 페이스에서 임베딩 모델 로드
embed_model_name = "MLP-KTLim/llama-3-Korean-Bllossom-8B"
embed_model = SentenceTransformer(embed_model_name)

In [ ]:
# 방법 1
# 임베딩 진행하면서 진행상황 확인
from tqdm import tqdm

embeddings = []
sentence = sentences[:100]
for sent in tqdm(sentence, desc="임베딩 중"):
    emb = embed_model.encode(sent)
    embeddings.append(emb)

In [ ]:
# 방법 2
# 문장을 배치 단위로 나눠서 처리 + 진행률 출력 : 더 빠름
batch_size = 32
embeddings = []

for i in tqdm(range(0, len(sentences), batch_size), desc="배치 임베딩 중"):
    batch = sentences[i:i+batch_size]
    emb = embed_model.encode(batch)
    embeddings.extend(emb)

In [ ]:
metadata = [{"text": s, "page": i+1} for i, s in enumerate(sentences)]

## 4. faiss DB

In [ ]:
import pickle

# numpy array로 변환
embeddings_np = np.array(embeddings).astype('float32')

# FAISS 인덱스 생성 (L2 거리 기반)
dim = embeddings_np.shape[1]
index = faiss.IndexFlatL2(dim)

# 벡터 추가
index.add(embeddings_np)


# 저장할 경로 설정
save_dir = "./faiss_data"

save_path = os.path.join(save_dir, "my_index.idx")

# 디렉토리 없으면 생성
os.makedirs(save_dir, exist_ok=True)

# 저장
faiss.write_index(index, save_path)
with open(os.path.join(save_dir, "metadata.pkl"), "wb") as f:
    pickle.dump(metadata, f)